**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Feb  9 18:29:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14691, done.
remote: Total 14691 (delta 0), reused 0 (delta 0), pack-reused 14691
Receiving objects: 100% (14691/14691), 13.25 MiB | 23.27 MiB/s, done.
Resolving deltas: 100% (10000/10000), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

#### 訓練前下面的Cell內容要做修改 (cfg設定檔)
- classes從80改成5(5是範例中的類別數，我們有5種臉要偵測)
- sed是取代工具，可以將前面的物件取代成後面的
  - ex: classes=80@classes=5@ 會把classes=80取代成classes=5
- @filters=255@filters=30是把filter的數量改成30，30並不是一個隨機值。計算公式為 (5 + n)*3
  - 其中n 相當於類別的數量 也就是classes的值

In [ ]:
!cp -f cfg/yolov3-tiny.cfg cfg/yolov3_training.cfg

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '135 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '177 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '127 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '171 s@filters=255@filters=18@' cfg/yolov3_training.cfg

#### 訓練前下面的Cell內容要做修改 (obj設定檔 * 2)
- classes從80改成5(範例中的類別數)
- obj.names裡面配置的是各個類別的名稱

In [ ]:
!echo -e "orange" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data


#### 下面的Cell執行後會開始下載預訓練的YOLO權重，會花一些時間

In [ ]:
# Download tiny-yolov3 weights darknet model 53
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2021-02-09 18:32:54--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M   348KB/s    in 1m 45s  

2021-02-09 18:34:40 (329 KB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
  inflating: data/obj/images/IMG_20201122_113154.txt  
  inflating: data/obj/images/IMG_20201122_113158.txt  
  inflating: data/obj/images/IMG_20201122_113207.txt  
  inflating: data/obj/images/IMG_20201122_113214.txt  
  inflating: data/obj/images/photo-1557800636-894a64c1696f.txt  
  inflating: data/obj/images/photo-1557800636-894a64c1696f.jpg  
  inflating: data/obj/images/IMG_20201122_113207.jpg  
  inflating: data/obj/images/IMG_20201122_113214.jpg  
  inflating: data/obj/images/IMG_20201122_113154.jpg  
  inflating: data/obj/images/IMG_20201122_113158.jpg  


In [ ]:
!cp /content/darknet/data/obj/images/photo-1557800636-894a64c1696f.jpg /content/darknet/test.jpg

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.530056 0.454861 0.383464 0.495370

['0', '0.530056', '0.454861', '0.383464', '0.495370']
0 data/obj/images/IMG_20201122_113214.txt
0 0.530056 0.454861 0.383464 0.495370
0 0.543403 0.490162 0.448351 0.636574

['0', '0.543403', '0.490162', '0.448351', '0.636574']
1 data/obj/images/IMG_20201122_113154.txt
0 0.543403 0.490162 0.448351 0.636574
0 0.510742 0.495081 0.315755 0.414931

['0', '0.510742', '0.495081', '0.315755', '0.414931']
2 data/obj/images/IMG_20201122_113207.txt
0 0.510742 0.495081 0.315755 0.414931
0 0.509000 0.625600 0.366000 0.259200

['0', '0.509000', '0.625600', '0.366000', '0.259200']
3 data/obj/images/photo-1557800636-894a64c1696f.txt
0 0.509000 0.625600 0.366000 0.259200
0 0.500000 0.417969 0.434028 0.543692

['0', '0.500000', '0.417969', '0.434028', '0.543692']
4 data/obj/images/IMG_20201122_113158.txt
0 0.500000 0.417969 0.434028 0.543692


In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/IMG_20201122_113207.jpg', 'data/obj/images/IMG_20201122_113158.jpg', 'data/obj/images/IMG_20201122_113214.jpg', 'data/obj/images/photo-1557800636-894a64c1696f.jpg', 'data/obj/images/IMG_20201122_113154.jpg']


In [ ]:
train_list = images_list[:-2]
test_list = images_list[-2:]


In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(train_list)) 
file.close() 

#Create test.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(test_list)) 
file.close() 


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg yolov3-tiny.weights -dont_show -clear

串流輸出內容已截斷至最後 5000 行。
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 224912, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.910349), count: 4, class_loss = 0.021706, iou_loss = 0.013713, total_loss = 0.035419 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 224916, rewritten_bbox = 0.000000 % 

 3522: 0.031928, 0.051580 avg loss, 0.001000 rate, 0.729475 seconds, 225408 images, 1.439957 hours left
Loaded: 7.912491 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.882981), count: 4, class_loss = 0.016364, iou_loss = 0.027184, total_loss = 0.043547 
v3 (mse loss, Normalizer: (

In [ ]:
%cd /content/darknet

In [ ]:
!mkdir -p /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo

In [ ]:
!cp -f /content/darknet/data/obj.names /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo
!cp -f /content/darknet/data/obj.data /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo
!cp -f /content/gdrive/My\ Drive/yolov3/yolov3_training_last.weights /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo
!cp -f /content/darknet/cfg/yolov3_training.cfg /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo
!cp -f /content/darknet/test.jpg /content/gdrive/My\ Drive/yolov3/orange_inference_config_file_tinyYolo

預測結果應該是 Tug

In [ ]:
!./darknet detector test ./data/obj.data ./cfg/yolov3_training.cfg /content/gdrive/My\ Drive/yolov3/yolov3_training_last.weights ./test.jpg